In [5]:
import google.generativeai as genai
import pandas as pd
import json
import os
import re
import time
from pathlib import Path

# --- 1. Configuración ---

# ¡IMPORTANTE! Configura tu clave API de forma segura.
# (No la escribas directamente aquí en un script de producción)
try:
    API_KEY = os.environ["GEMINI_API_KEY"] 
    genai.configure(api_key=API_KEY)
except KeyError:
    print("Error: Configura la variable de entorno 'GEMINI_API_KEY'")
    exit()

# -------------------------------
# 2. List available models
# -------------------------------
try:
    models = genai.list_models()
    print("Available Google Generative AI Models:\n")
    for model in models:
        print(f"- {model.name} | Supported input: {model.input_token_limit} tokens")
except Exception as e:
    print(f"Error listing models: {e}")


Available Google Generative AI Models:

- models/embedding-gecko-001 | Supported input: 1024 tokens
- models/gemini-2.5-pro-preview-03-25 | Supported input: 1048576 tokens
- models/gemini-2.5-flash-preview-05-20 | Supported input: 1048576 tokens
- models/gemini-2.5-flash | Supported input: 1048576 tokens
- models/gemini-2.5-flash-lite-preview-06-17 | Supported input: 1048576 tokens
- models/gemini-2.5-pro-preview-05-06 | Supported input: 1048576 tokens
- models/gemini-2.5-pro-preview-06-05 | Supported input: 1048576 tokens
- models/gemini-2.5-pro | Supported input: 1048576 tokens
- models/gemini-2.0-flash-exp | Supported input: 1048576 tokens
- models/gemini-2.0-flash | Supported input: 1048576 tokens
- models/gemini-2.0-flash-001 | Supported input: 1048576 tokens
- models/gemini-2.0-flash-exp-image-generation | Supported input: 1048576 tokens
- models/gemini-2.0-flash-lite-001 | Supported input: 1048576 tokens
- models/gemini-2.0-flash-lite | Supported input: 1048576 tokens
- models/g

In [7]:
MODEL_TO_USE = 'models/gemini-flash-latest'

# Configuración del modelo
model = genai.GenerativeModel(MODEL_TO_USE)

# --- 2. Definición de Rutas (Paths) ---
# Usa las rutas que especificaste
BASE_PATH = Path(r"D:\github\plnTallerRAG\data")
CORPUS_PATH = BASE_PATH / "corpus"
CHUNKS_CSV_PATH = BASE_PATH / "chunks_debug.csv"
OUTPUT_JSON_PATH = BASE_PATH / "gold_standard_dataset.json"

# --- 3. El "Meta-Prompt" (La clave del éxito) ---
# Este prompt le pide al modelo que actúe como un experto en evaluación
# y que genere las preguntas, respuestas y *pasajes relevantes*.
# --- 3. El "Meta-Prompt" (Corregido) ---
# Este prompt le pide al modelo que actúe como un experto en evaluación
# y que genere las preguntas, respuestas y *pasajes relevantes*.
PROMPT_TEMPLATE = """
Eres un experto en crear 'Padrón de Oro' (Gold Standard) para evaluar sistemas de Búsqueda Aumentada por Generación (RAG).
Tu tarea es leer el texto de un capítulo de un libro y generar {num_preguntas} preguntas de alta calidad que pongan a prueba la comprensión y la capacidad de síntesis de un sistema de búsqueda.

Para cada pregunta, debes proporcionar:
1.  `query`: La pregunta en sí. Debe ser clara y requerir información específica del texto.
2.  `ideal_answer`: Una respuesta concisa, objetiva y en lenguaje natural, basada ÚNICAMENTE en el texto proporcionado.
3.  `relevant_passages`: Una lista de los fragmentos de texto (pasajes) VERBATIM y EXACTOS del capítulo que contienen la información necesaria para responder la pregunta. Estos pasajes deben ser lo más cortos posible pero contener la evidencia completa.

Restricciones Importantes:
-   Genera preguntas que no se respondan con un simple "sí" o "no".
-   Genera preguntas que requieran conectar información de, potencialmente, varias partes del texto.
-   La respuesta JSON debe ser una lista de diccionarios, sin ningún otro texto introductorio o de cierre.

Devuelve el resultado como una lista JSON válida.

--- INICIO DEL TEXTO DEL CAPÍTULO ---
{chapter_text}
--- FIN DEL TEXTO DEL CAPÍTULO ---

Aquí está tu formato de salida JSON (una lista de diccionarios):
[
  {{  
    "query": "...",
    "ideal_answer": "...",
    "relevant_passages": ["...", "..."]
  }}, <-- ¡CORREGIDO!
  ...
]
"""

# --- 4. Funciones Auxiliares ---

def cargar_chunks(csv_path):
    """Carga el archivo CSV de chunks en un DataFrame de Pandas."""
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
        print(f"Cargados {len(df)} chunks desde '{csv_path}'")
        return df
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo de chunks en '{csv_path}'")
        exit()
    except Exception as e:
        print(f"Error al leer el CSV: {e}")
        exit()

def llamar_api_gemini(prompt_completo):
    """Envía el prompt a la API de Gemini y parsea la respuesta JSON."""
    try:
        response = model.generate_content(prompt_completo)
        
        # Limpiar la respuesta para extraer solo el JSON
        # (A veces la API envuelve la respuesta en ```json ... ```)
        json_match = re.search(r'\[.*\]', response.text, re.DOTALL)
        if not json_match:
            print("Error: La respuesta de la API no contenía un JSON válido.")
            print("Respuesta recibida:", response.text)
            return None

        json_string = json_match.group(0)
        return json.loads(json_string)
    
    except json.JSONDecodeError as e:
        print(f"Error al decodificar el JSON de la API: {e}")
        print("Respuesta recibida:", response.text)
        return None
    except Exception as e:
        print(f"Error en la llamada a la API: {e}")
        return None

def encontrar_chunks_relevantes(passages, chapter_chunks_df):
    """
    Mapea los pasajes de texto a los chunk_ids.
    Esta es la parte más delicada.
    """
    relevant_ids = set()
    for passage in passages:
        p_clean = passage.strip()
        if not p_clean:
            continue
        
        # Escapamos caracteres especiales para la búsqueda
        p_regex = re.escape(p_clean)

        # Buscar chunks que *contengan* el pasaje.
        # na=False es importante si hay valores NaN en 'text_content'
        try:
            matches = chapter_chunks_df[chapter_chunks_df['text_content'].str.contains(p_regex, na=False)]
            for chunk_id in matches['chunk_id']:
                relevant_ids.add(chunk_id)
        except Exception as e:
            print(f"Error durante la búsqueda de regex en el DataFrame: {e}")

    return sorted(list(relevant_ids))

# --- 5. Función Principal ---

def main():
    print("Iniciando la generación del 'Gold Standard'...")
    
    df_chunks = cargar_chunks(CHUNKS_CSV_PATH)
    
    gold_standard_dataset = []
    
    # Listar todos los archivos de texto en la carpeta corpus
    archivos_corpus = sorted(list(CORPUS_PATH.glob("*.txt")))
    total_archivos = len(archivos_corpus)
    
    print(f"Se encontraron {total_archivos} capítulos en '{CORPUS_PATH}'")

    for i, chapter_file_path in enumerate(archivos_corpus):
        chapter_filename = chapter_file_path.name
        print(f"\n--- Procesando Capítulo {i+1}/{total_archivos}: {chapter_filename} ---")
        
        try:
            with open(chapter_file_path, 'r', encoding='utf-8') as f:
                chapter_text = f.read()
        except Exception as e:
            print(f"Error al leer el archivo {chapter_filename}: {e}")
            continue
            
        # 1. Preparar el prompt para este capítulo
        prompt_completo = PROMPT_TEMPLATE.format(
            num_preguntas=3, # Pides 3 preguntas por capítulo
            chapter_text=chapter_text
        )
        
        # 2. Llamar a la API
        print("Enviando solicitud a la API de Gemini...")
        generated_data = llamar_api_gemini(prompt_completo)
        
        if not generated_data:
            print(f"No se pudo generar datos para {chapter_filename}. Saltando.")
            continue
            
        print(f"API respondió con {len(generated_data)} preguntas.")
        
        # 3. Filtrar los chunks relevantes para este capítulo
        chapter_chunks_df = df_chunks[df_chunks['source_document'] == chapter_filename].copy()
        if chapter_chunks_df.empty:
            print(f"Advertencia: No se encontraron chunks para {chapter_filename} en el CSV.")
            continue
            
        # 4. Mapear pasajes a chunks para cada pregunta generada
        for qa_item in generated_data:
            query = qa_item.get("query")
            ideal_answer = qa_item.get("ideal_answer")
            passages = qa_item.get("relevant_passages", [])
            
            if not all([query, ideal_answer, passages]):
                print("Advertencia: Item de QA incompleto. Saltando.")
                continue

            # 5. Encontrar los chunk_ids
            relevant_chunk_ids = encontrar_chunks_relevantes(passages, chapter_chunks_df)
            
            if not relevant_chunk_ids:
                print(f"Advertencia: No se pudo mapear ningún chunk para la query: '{query[:50]}...'")
                # Opcional: podrías decidir no incluirlo si no hay chunks
                # continue
            
            # 6. Ensamblar el item del "Gold Standard"
            gold_standard_item = {
                "query": query,
                "relevant_chunk_ids": relevant_chunk_ids,
                "ideal_answer": ideal_answer,
                "source_chapter": chapter_filename # Añadir esto es buena idea
            }
            gold_standard_dataset.append(gold_standard_item)
            print(f"  + Query agregada: '{query[:70]}...' (Chunks: {relevant_chunk_ids})")

        # Pausa para evitar exceder los límites de la API
        time.sleep(2) # Ajusta según sea necesario

    # --- 6. Guardar el Resultado Final ---
    print(f"\nProcesamiento completado. Total de {len(gold_standard_dataset)} preguntas generadas.")
    
    try:
        with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
            json.dump(gold_standard_dataset, f, indent=2, ensure_ascii=False)
        print(f"¡Éxito! 'Gold Standard' guardado en: '{OUTPUT_JSON_PATH}'")
    except Exception as e:
        print(f"Error al guardar el archivo JSON final: {e}")

if __name__ == "__main__":
    main()

Iniciando la generación del 'Gold Standard'...
Cargados 1351 chunks desde 'D:\github\plnTallerRAG\data\chunks_debug.csv'
Se encontraron 36 capítulos en 'D:\github\plnTallerRAG\data\corpus'

--- Procesando Capítulo 1/36: 01-prólogo.txt ---
Enviando solicitud a la API de Gemini...
API respondió con 3 preguntas.
  + Query agregada: '¿Cuáles son los tres pilares del Sistema Integral para la Paz establec...' (Chunks: ['01-prólogo.txt_0003', '01-prólogo.txt_0004'])
  + Query agregada: '¿Cuál fue la pregunta macro que orientó la investigación de la Comisió...' (Chunks: ['01-prólogo.txt_0013', '01-prólogo.txt_0014', '01-prólogo.txt_0017', '01-prólogo.txt_0018'])
  + Query agregada: '¿De cuántos tomos y qué tipo de documento complementario consta el inf...' (Chunks: ['01-prólogo.txt_0019'])

--- Procesando Capítulo 2/36: 02-introducción.txt ---
Enviando solicitud a la API de Gemini...
API respondió con 3 preguntas.
Advertencia: No se pudo mapear ningún chunk para la query: '¿Qué tipos de inform